Импорт библиотек

In [ ]:
pip install catboost

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
import catboost as cb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns

Синтезирование данных (Если есть свои -- пропускайте)

In [16]:
def generate_synthetic_data(num_samples=1000):
    np.random.seed(42)
    data = pd.DataFrame({
        'Age': np.random.randint(18, 70, size=num_samples),
        'Balance': np.random.uniform(0, 100000, size=num_samples),
        'NumOfProducts': np.random.randint(1, 4, size=num_samples),
        'HasCrCard': np.random.randint(0, 2, size=num_samples),
        'IsActiveMember': np.random.randint(0, 2, size=num_samples),
        'EstimatedSalary': np.random.uniform(10000, 150000, size=num_samples),
        'Churn': np.random.randint(0, 2, size=num_samples)
    })
    return data

# Генерация данных
data = generate_synthetic_data()
data.to_csv('../data/generated_data.csv', index=False)
import os


Предобработка данных

In [18]:
data = pd.read_csv('../data/generated_data.csv')

# Преобразование категориальных переменных
data = pd.get_dummies(data, drop_first=True)

# Разделение данных
X = data.drop('Churn', axis=1)
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Модель Catboost

In [21]:
catboost_model = cb.CatBoostClassifier(verbose=0)
catboost_model.fit(X_train, y_train)
y_pred_cb = catboost_model.predict(X_test)

print(f"CatBoost Accuracy: {accuracy_score(y_test, y_pred_cb)}")
print(f"CatBoost Precision: {precision_score(y_test, y_pred_cb)}")
print(f"CatBoost Recall: {recall_score(y_test, y_pred_cb)}")


CatBoost Accuracy: 0.485
CatBoost Precision: 0.4827586206896552
CatBoost Recall: 0.42


Модель TensorFlow

In [26]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)

y_pred_tf = (model.predict(X_test_scaled) > 0.5).astype("int32")

print(f"TensorFlow Accuracy: {accuracy_score(y_test, y_pred_tf)}")
print(f"TensorFlow Precision: {precision_score(y_test, y_pred_tf)}")
print(f"TensorFlow Recall: {recall_score(y_test, y_pred_tf)}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
TensorFlow Accuracy: 0.48
TensorFlow Precision: 0.4772727272727273
TensorFlow Recall: 0.42


In [ ]:
catboost_model.save_model('../models/catboost_model.cbm')
model.save('../models/model.h5')
